## Kelley hu4d5-5 VH-F100A error analysis

In this notebook we will use a random forest model to find the most energetically influential degrees of freedom for the VH-F100A TI production run. Next, we will compare the sampling of these DOF during TI production to a free energy profile derived from end state GaMD sampling. We will attempt to  correct any inaccurate sampling in the TI data and find the estimated ddG before and after the correction. 

In [1]:
from common_functions import *

### Ingesting initial TI data

In [11]:
os.chdir("./TI_data/VH-F100A/")
geom_dvdls = pd.read_csv("F100A_bound.csv")
geom_dvdls_ub = pd.read_csv("F100A_unbound.csv")

### Original ddG estimate:

In [9]:
orig_dG_bd = geom_dvdls.groupby("Lambda").mean().sum()["weight_dvdl"]
orig_dG_ubd = geom_dvdls_ub.groupby("Lambda").mean().sum()["weight_dvdl"]

empirical_value = 1.16
orig_error = abs((orig_dG_bd - orig_dG_ubd) - empirical_value)

print("Original ddG estimate: ")
print(f"{round(orig_dG_bd - orig_dG_ubd, 4)} kcal/mol ")
print()
print("Original ddG error: ")
print(f"{round(orig_error, 4)} kcal/mol")

Original ddG estimate: 
1.6678 kcal/mol 

Original ddG error: 
0.5078 kcal/mol


### Removing correlated variables among candidate degrees of freedom

This is important because we want to limit the noise in our model training. See our methods/supplemental methods section for our process to choose the input features.

In [11]:
X = geom_dvdls.drop([
    "#Frame", "weight_dvdl", "dvdl", "Run", "Lambda", "Y184_chi2", "E190_chi2", "D457_chi2"
], axis=1)

Y = geom_dvdls["weight_dvdl"]

X_scl = pd.DataFrame(StandardScaler().fit_transform(X))
X_scl.columns = X.columns


### Checking to see if there is any cross-correlations within the independent variables

In [12]:
absCorr = abs(X_scl.corr())
for i in absCorr.columns:
    for j in absCorr.index:
        cor = absCorr.loc[i, j]
        if abs(cor) > 0.5 and i != j:
            print(i, j)
            print(cor)
            

### Using random forest model to identify the most energetically influential degrees of freedom

We run our model 25 times, then sort the results by the mean of feature importance across the 25 iterations. For this particular perturbation, the model $R^2$ between the geometric DOF (nearby side chain rotamers or interatomic distances) and the energetic DV/DL was not strong enough for us to check the sampling of these degrees of freedom. 

In [13]:
rfeDefault = RFE(estimator=DecisionTreeRegressor(max_depth=5, random_state=42), n_features_to_select=0.75, step=0.05)
rfDefault = RandomForestRegressor(
    max_depth=10, n_estimators=200, oob_score=True, max_features=0.6, min_samples_leaf = 7, min_samples_split=14, random_state=42
)

pipelineDefault_rf = Pipeline([
    ('feature_scaling', StandardScaler()),
    ('feature_selection', rfeDefault),
    ('regression_model', rfDefault)
])


imps = benchmark_model(pipelineDefault_rf, X_scl, Y, geom_dvdls["Lambda"])
imps[["Mean", "Median"]].sort_values(by="Mean", ascending=False)[:15]

Avg. training r2: 
0.4987
Training r2 std dev: 
0.0014
Avg. test r2: 
0.2885
Testing r2 std dev: 
0.0055


,Mean,Median
Y101_chi2,0.099989,0.116101
Y101_chi1,0.097042,0.090523
F453_chi2,0.080770,0.107268
F188_chi2,0.068015,0.081607
W448_chi2,0.067131,0.070912
F453_chi1,0.062616,0.070289
Y184_chi1,0.054163,0.063798
F188_chi1,0.053156,0.047592
D451_chi1,0.051847,0.034554
W448_chi1,0.047542,0.049317
